In [1]:
from network_evaluation_tools import gene_conversion_tools as gct
from network_evaluation_tools import data_import_tools as dit
import pandas as pd
import itertools
import time
import matplotlib.pyplot as plt

In [2]:
wd = '/cellar/users/snwright/Data/Network_Analysis/'

## TBD

There seem to be many changes in the data and I need to figure out the best source. It is difficult to access past version of Reactome. Need to figure out which is most appropriate for this task. 

From PathwayCommons:
* v12: v69
* v11: v67
* v9: v61
* v8: v56
Maybe see how closely v61 matches original data?

Reactome.org is currently at v78

# Version 2.0
## Load Reactome Raw Data
#### Source: http://www.reactome.org/download/current/homo_sapiens.interactions.txt.gz
#### File to download: The link labelled "Human protein-protein interaction pairs in tab-delimited format" seems to have many more interactions than the MITAB file format. This is the file that we will use for this network.
Downloaded: Nov 16, 2021   
Last Updated: Oct 10, 2021

In [60]:

Reactome_Raw_v3 = pd.read_csv(wd+'Network_Data_Raw/Reactome/PathwayCommons12.reactome.hgnc.txt.gz',sep='\t',skiprows=1, header=None, low_memory=False)
print('Raw Edges in Reactome v69:', len(Reactome_Raw_v3))

Raw Edges in Reactome v69: 365961


In [61]:
Reactome_Raw_v3.head()

,0,1,2,3,4,5,6
0,A1CF,in-complex-with,APOBEC1,Reactome,NaN,Formation of the Editosome;mRNA Editing: C to ...,http://pathwaycommons.org/pc12/Complex_5987964...
1,A1CF,in-complex-with,APOBEC2,Reactome,NaN,Formation of the Editosome;mRNA Editing: C to ...,http://pathwaycommons.org/pc12/Complex_5987964...
2,A1CF,in-complex-with,APOBEC3A,Reactome,NaN,Formation of the Editosome;mRNA Editing: C to ...,http://pathwaycommons.org/pc12/Complex_5987964...
3,A1CF,in-complex-with,APOBEC3B,Reactome,NaN,Formation of the Editosome;mRNA Editing: C to ...,http://pathwaycommons.org/pc12/Complex_5987964...
4,A1CF,in-complex-with,APOBEC3C,Reactome,NaN,Formation of the Editosome;mRNA Editing: C to ...,http://pathwaycommons.org/pc12/Complex_5987964...


In [63]:
Reactome_Raw_v3[1].value_counts()

in-complex-with                               141797
controls-state-change-of                      116870
catalysis-precedes                             52989
chemical-affects                               13072
ProteinReference                                9858
controls-production-of                          5343
controls-transport-of                           5325
consumption-controlled-by                       5173
controls-expression-of                          3757
controls-phosphorylation-of                     3415
used-to-produce                                 3369
SmallMoleculeReference                          1957
controls-transport-of-chemical                  1927
reacts-with                                      599
DnaReference;ProteinReference                    435
DnaReference                                      33
DnaReference;ProteinReference;RnaReference        22
ProteinReference;RnaReference                     15
DnaReference;RnaReference                     

In [52]:

Reactome_Raw_v2 = pd.read_csv(wd+'Network_Data_Raw/Reactome/Reactome_v78_interactions.txt',sep='\t',skiprows=1, header=None, low_memory=False)
print('Raw Edges in Reactome v78:', len(Reactome_Raw_v2))

Raw Edges in Reactome v78: 101646


In [53]:
Reactome_Raw_v2.head()

,0,1,2,3,4,5,6,7,8
0,uniprotkb:Q9Y287,ENSEMBL:ENST00000378549|ENSEMBL:ENSP0000049722...,-,uniprotkb:Q9Y287,ENSEMBL:ENST00000378549|ENSEMBL:ENSP0000049722...,-,physical association,reactome:R-HSA-976871,14690516|10391242
1,uniprotkb:P37840,ENSEMBL:ENST00000506244|ENSEMBL:ENST0000039499...,-,uniprotkb:P37840,ENSEMBL:ENST00000506244|ENSEMBL:ENST0000039499...,-,physical association,reactome:R-HSA-1247852,24243840
2,uniprotkb:P0DJI8,ENSEMBL:ENST00000356524|ENSEMBL:ENSG0000017343...,-,uniprotkb:P0DJI8,ENSEMBL:ENST00000356524|ENSEMBL:ENSG0000017343...,-,physical association,reactome:R-HSA-976898,19393650|103558
3,uniprotkb:P06727,ENSEMBL:ENST00000357780|ENSEMBL:ENSG0000011024...,-,uniprotkb:P06727,ENSEMBL:ENST00000357780|ENSEMBL:ENSG0000011024...,-,physical association,reactome:R-HSA-976889,15146166
4,uniprotkb:P01160,ENSEMBL:ENST00000376480|ENSEMBL:ENSP0000036566...,-,uniprotkb:P01160,ENSEMBL:ENST00000376480|ENSEMBL:ENSP0000036566...,-,physical association,reactome:R-HSA-976987,2142465|2945573


In [57]:
Reactome_Raw_v2[6].value_counts()

physical association                                  91275
enzymatic reaction                                     7285
cleavage reaction                                      1437
dephosphorylation reaction                              603
oxidoreductase activity electron transfer reaction      398
phospholipase reaction                                  142
glycosylation reaction                                   96
acetylation reaction                                     94
isomerase reaction                                       86
nucleoside triphosphatase reaction                       52
gtpase reaction                                          48
deubiquitination reaction                                24
demethylation reaction                                   20
decarboxylation reaction                                 19
deacetylation reaction                                   18
carboxylation reaction                                   16
amidation reaction                      

In [55]:
# Get edge list of network (filter for duplicate edges and self-edges)
query_edgelist_filt_v2 = Reactome_Raw_v2.drop_duplicates(subset = [0, 3])
print(len(query_edgelist_filt_v2), "Raw Reactome Edges after removing duplicate edges")
query_edgelist_filt2_v2 = query_edgelist_filt_v2[query_edgelist_filt_v2[0]!=query_edgelist_filt_v2[2]]
print(len(query_edgelist_filt2_v2), "Raw Reactome Edges after removing duplicate and self-edges")
query_edgelist_v2 = query_edgelist_filt2_v2.values.tolist()

40772 Raw Reactome Edges after removing duplicate edges
40772 Raw Reactome Edges after removing duplicate and self-edges


In [49]:
# Extract gene list
Reactome_Raw_Genes_v2 = list(set(query_edgelist_filt2_v2[0]).union(set(query_edgelist_filt2_v2[3])))

## Convert Genes from UniProtKB to Symbol

In [50]:
query_string_v2, valid_genes_v2, invalid_genes_v2 = gct.query_constructor(Reactome_Raw_Genes_v2)

14279 Valid Query Genes
10259 Invalid Query Genes


TypeError: sequence item 18: expected str instance, NoneType found

In [33]:
# Set scopes (gene naming systems to search)
scopes = "uniprot"

# Set fields (systems from which to return gene names from)
fields = "symbol, entrezgene"

# Query MyGene.Info
match_list_v2 = gct.query_batch(query_string_v2, scopes=scopes, fields=fields)


100%|██████████| 9/9 [00:06<00:00,  1.33it/s]

8375 Matched query results
Batch query complete: 6.78 seconds


In [34]:
# Original 371 without full matching results, 102 with multiple matches
match_table_trim_v2, query_to_symbol_v2, query_to_entrez_v2 = gct.construct_query_map_table(match_list_v2, valid_genes_v2)

Queries without full matching results found: 2187

86 Queries with mutliple matches found

Query mapping table/dictionary construction complete: 2.15 seconds


## Construct Converted Network

In [35]:
# Format edge list by removing prefixes from all interactors
query_edgelist_fmt_v2 = [[gct.get_identifier_without_prefix(edge[0]), gct.get_identifier_without_prefix(edge[1])] for edge in query_edgelist_v2]

In [36]:
# Convert network edge list to symbol
Reactome_edgelist_symbol_v2 = gct.convert_edgelist(query_edgelist_fmt_v2, query_to_symbol_v2, weighted=False)

In [37]:

Reactome_edgelist_symbol_filt_v2 = gct.filter_converted_edgelist(Reactome_edgelist_symbol_v2, weighted=False)

38812 input edges
3 self-edges removed
15351 edges with un-mapped genes removed
87 duplicate edges removed
Edge list filtered: 0.01 seconds
23371 Edges remaining


In [38]:
# Save filtered, converted edge list to file
gct.write_edgelist(Reactome_edgelist_symbol_filt_v2, wd+'Processed_Data/Network_SIFs_Symbol/Reactome_Symbol_v78.sif', binary=True)

Edge list saved: 0.04 seconds


# Version 1.0 (original)
## Load Reactome Raw Data
#### Source: http://www.reactome.org/download/current/homo_sapiens.interactions.txt.gz
#### File to download: The link labelled "Human protein-protein interaction pairs in tab-delimited format" seems to have many more interactions than the MITAB file format. This is the file that we will use for this network.
Downloaded: June 15, 2017    
Last Updated: April 20, 2017  

In [21]:

Reactome_Raw = pd.read_csv(wd+'Network_Data_Raw/Reactome/Reactome_v60.interactions.txt',sep='\t',skiprows=1, header=None, low_memory=False)
print('Raw Edges in Reactome v60:', len(Reactome_Raw))

Raw Edges in Reactome v60: 2523567


In [58]:
Reactome_Raw.head()

,0,1,2,3,4,5,6,7,8
0,UniProt:A0A075B6P5,NaN,NaN,UniProt:A0A075B6P5,NaN,NaN,neighbouring_reaction,R-HSA-166753.2<->R-HSA-173631.3,NaN
1,UniProt:A0A075B6P5,NaN,NaN,UniProt:A0A075B6P5,NaN,NaN,neighbouring_reaction,R-HSA-173626.4<->R-HSA-173631.3,NaN
2,UniProt:A0A075B6P5,NaN,NaN,UniProt:A0A075B6P5,NaN,NaN,neighbouring_reaction,R-HSA-173631.3<->R-HSA-166753.2,NaN
3,UniProt:A0A075B6P5,NaN,NaN,UniProt:A0A075B6P5,NaN,NaN,neighbouring_reaction,R-HSA-173631.3<->R-HSA-173626.4,NaN
4,UniProt:A0A075B6P5,NaN,NaN,UniProt:A0A075B6P5,NaN,NaN,neighbouring_reaction,R-HSA-173631.3<->R-HSA-8852266.1,NaN


In [59]:
Reactome_Raw[6].value_counts()

neighbouring_reaction    1878645
reaction                  348127
indirect_complex          245850
direct_complex             50945
Name: 6, dtype: int64

In [4]:
# Get edge list of network (filter for duplicate edges and self-edges)
query_edgelist_filt = Reactome_Raw[[0,3]].drop_duplicates()
print(len(query_edgelist_filt), "Raw Reactome Edges after removing duplicate edges")
query_edgelist_filt2 = query_edgelist_filt[query_edgelist_filt[0]!=query_edgelist_filt[3]]
print(len(query_edgelist_filt2), "Raw Reactome Edges after removing duplicate and self-edges")
query_edgelist = query_edgelist_filt2.values.tolist()

214432 Raw Reactome Edges after removing duplicate edges
210066 Raw Reactome Edges after removing duplicate and self-edges


In [6]:
# Extract gene list
Reactome_Raw_Genes = list(set(query_edgelist_filt2[0]).union(set(query_edgelist_filt2[3])))

## Convert Genes from UniProtKB to Symbol

In [7]:
query_string, valid_genes, invalid_genes = gct.query_constructor(Reactome_Raw_Genes)

8387 Valid Query Genes
0 Invalid Query Genes


In [8]:
# Set scopes (gene naming systems to search)
scopes = "uniprot"

# Set fields (systems from which to return gene names from)
fields = "symbol, entrezgene"

# Query MyGene.Info
match_list = gct.query_batch(query_string, scopes=scopes, fields=fields)


100%|██████████| 9/9 [00:09<00:00,  1.06s/it]

8539 Matched query results
Batch query complete: 9.82 seconds


In [9]:
# Original 371 without full matching results, 102 with multiple matches
match_table_trim, query_to_symbol, query_to_entrez = gct.construct_query_map_table(match_list, valid_genes)

Queries without full matching results found: 371

99 Queries with mutliple matches found

Query mapping table/dictionary construction complete: 2.27 seconds


## Construct Converted Network

In [10]:
# Format edge list by removing prefixes from all interactors
query_edgelist_fmt = [[gct.get_identifier_without_prefix(edge[0]), gct.get_identifier_without_prefix(edge[1])] for edge in query_edgelist]

In [11]:
# Convert network edge list to symbol
Reactome_edgelist_symbol = gct.convert_edgelist(query_edgelist_fmt, query_to_symbol, weighted=False)

In [13]:
# Original
# 210066 input edges
# 2708 self-edges removed
# 10886 edges with un-mapped genes removed
# 1970 duplicate edges removed
# Edge list filtered: 0.51 seconds
# 194502 Edges remaining
# Filter converted edge list
Reactome_edgelist_symbol_filt = gct.filter_converted_edgelist(Reactome_edgelist_symbol, weighted=False)

210066 input edges
6 self-edges removed
13993 edges with un-mapped genes removed
318 duplicate edges removed
Edge list filtered: 0.06 seconds
195749 Edges remaining


In [14]:
# Save filtered, converted edge list to file
gct.write_edgelist(Reactome_edgelist_symbol_filt, wd+'Processed_Data/Network_SIFs_Symbol/Reactome_Symbol_2017.sif', binary=True)

Edge list saved: 0.25 seconds
